In [ ]:
# pip install yellowbrick
# pour pep8
#!pip install pycodestyle pycodestyle_magic
# %load_ext pycodestyle_magic
#!pip install autopep8
# https://stackoverflow.com/questions/47784440/how-can-i-solve-error-in-jupyter-notebook
#%load_ext pycodestyle_magic 
#%pycodestyle_off

# IMPORT

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

"""Import des fichiers olist."""


geo = pd.read_csv('input/archive/olist_geolocation_dataset.csv',
                  delimiter=',',
                  error_bad_lines=False,
                  low_memory=False)
cust = pd.read_csv('input/archive/olist_customers_dataset.csv',
                   delimiter=',',
                   error_bad_lines=False,
                   low_memory=False)
order_payment = pd.read_csv('input/archive/olist_order_payments_dataset.csv',
                            delimiter=',',
                            error_bad_lines=False,
                            low_memory=False)
order_review = pd.read_csv('input/archive/olist_order_reviews_dataset.csv',
                           delimiter=',',
                           error_bad_lines=False,
                           low_memory=False)
order = pd.read_csv('input/archive/olist_orders_dataset.csv',
                    delimiter=',',
                    error_bad_lines=False,
                    low_memory=False)

order_item = pd.read_csv('input/archive/olist_order_items_dataset.csv',
                         delimiter=',',
                         error_bad_lines=False,
                         low_memory=False)
product = pd.read_csv('input/archive/olist_products_dataset.csv',
                      delimiter=',',
                      error_bad_lines=False,
                      low_memory=False)
seller = pd.read_csv('input/archive/olist_sellers_dataset.csv',
                     delimiter=',',
                     error_bad_lines=False,
                     low_memory=False)
product_cat = pd.read_csv(
    'input/archive/product_category_name_translation.csv',
    delimiter=',',
    error_bad_lines=False,
    low_memory=False)

# Jointure product,product cat, seller, order_item

In [ ]:
"""Jointure des tables: product, product cat, seller, order_item."""

##############################################################################
# fusion product/cat anglais
# cat englais
product2 = pd.merge(product,
                    product_cat, how='outer',
                    on='product_category_name')
# volume
product2['vol'] = product2.product_length_cm * \
    product2.product_height_cm*product2.product_width_cm*(1e-6)
# drop
product2 = product2.drop(columns=['product_category_name',
                                  'product_name_lenght',
                                  'product_length_cm',
                                  'product_height_cm',
                                  'product_width_cm'])
product2.columns = ['product_id',
                    'prod_desc_l',
                    'prod_foto_nb',
                    'prod_w',
                    'prod_cat',
                    'prod_vol']

##############################################################################
# fusion seller (adresse)/order_item/product
product3 = order_item.set_index('seller_id').join(
    seller.set_index('seller_id'))
product3.columns = ['order_id',
                    'order_item_id',
                    'product_id',
                    'ord_item_ship_lim_dat',
                    'ord_item_price',
                    'ord_item_fdp',
                    'sel_zip',
                    'sel_city',
                    'sel_state']

##############################################################################
# fusion product/cat anglais et seller (adresse)/order_item/product
product3 = product3.reset_index().set_index(
    'product_id').join(product2.set_index('product_id'))

# Fusion order_payment/order_review/cust/order/geo

In [ ]:
import datetime as dt

"""Jointure des tables olist: order et order_payment."""

pay1 = order_payment.groupby(by="order_id").agg({
    'payment_sequential': ['max'],
    'payment_installments': ['max'],
    'payment_value': ['sum']})
pay1.columns = pay1.columns.droplevel(0)
pay1.columns = ['pay_seq', 'pay_ins', 'pay_val']

##############################################################################
# calcul des montant par type de paiement
pay4 = order_payment.loc[
    order_payment.payment_type == 'credit_card', :].groupby(
    by="order_id").agg({'payment_value': ['sum']})
pay4.columns = pay4.columns.droplevel(0)
pay4.columns = ['pay_val_creditc']
pay5 = order_payment.loc[
    order_payment.payment_type == 'debit_card', :].groupby(
    by="order_id").agg({'payment_value': ['sum']})
pay5.columns = pay5.columns.droplevel(0)
pay5.columns = ['pay_val_debitc']
pay6 = order_payment.loc[
    order_payment.payment_type == 'boleto', :].groupby(
    by="order_id").agg({'payment_value': ['sum']})
pay6.columns = pay6.columns.droplevel(0)
pay6.columns = ['pay_val_cash']
pay7 = order_payment.loc[
    order_payment.payment_type == 'voucher', :].groupby(
    by="order_id").agg({'payment_value': ['sum']})
pay7.columns = pay7.columns.droplevel(0)
pay7.columns = ['pay_val_voucher']
pay8 = order_payment.loc[
    order_payment.payment_type == 'not_defined', :].groupby(
    by="order_id").agg({'payment_value': ['sum']})
pay8.columns = pay8.columns.droplevel(0)
pay8.columns = ['pay_val_nd']

pay1 = pay1.join(pay4)
pay1 = pay1.join(pay5)
pay1 = pay1.join(pay6)
pay1 = pay1.join(pay7)
pay1 = pay1.join(pay8)

pay1[['pay_val_creditc', 'pay_val_debitc', 'pay_val_cash',
      'pay_val_voucher', 'pay_val_nd']] = pay1[[
          'pay_val_creditc', 'pay_val_debitc', 'pay_val_cash',
          'pay_val_voucher', 'pay_val_nd']].fillna(0)
pay1['pay_val_pct_creditc'] = pay1['pay_val_creditc']/pay1['pay_val']
pay1['pay_val_pct_debitc'] = pay1['pay_val_debitc']/pay1['pay_val']
pay1['pay_val_pct_cash'] = pay1['pay_val_cash']/pay1['pay_val']
pay1['pay_val_pct_voucher'] = pay1['pay_val_voucher']/pay1['pay_val']
pay1['pay_val_pct_nd'] = pay1['pay_val_nd']/pay1['pay_val']

##############################################################################
# petit nettoyage
pay1 = pay1.loc[pay1.pay_val_pct_nd != 1, :]
pay1 = pay1.drop(columns=['pay_val_nd', 'pay_val_pct_nd'])

pay1.loc[pay1.pay_val_pct_cash == 1, ['pay_type']] = 'cash'
pay1.loc[pay1.pay_val_pct_debitc == 1, ['pay_type']] = 'debit'
pay1.loc[((pay1.pay_val_pct_creditc == 1) & (
    pay1.pay_val_pct_voucher == 0)), ['pay_type']] = 'credit'
pay1.loc[((pay1.pay_val_pct_voucher == 1) & (
    pay1.pay_val_pct_creditc == 0)), ['pay_type']] = 'voucher'
pay1.loc[((pay1.pay_val_pct_voucher > 0) & (pay1.pay_val_pct_creditc > 0)), [
    'pay_type']] = 'mix credit voucher'

##############################################################################
# autre nettoyage pour données inutiles
pay1 = pay1.drop(columns=['pay_val_pct_creditc',
                 'pay_val_pct_debitc', 'pay_val_pct_cash'])
pay1 = pay1.drop(columns=['pay_val_creditc',
                          'pay_val_debitc', 'pay_val_cash'])

##############################################################################
# on fusionne avec order
order1 = order.set_index('order_id')
order1 = order1.join(pay1)

##############################################################################
# "datetime"
order1.order_purchase_timestamp = pd.to_datetime(
    order1.order_purchase_timestamp, errors='coerce')
order1.order_approved_at = pd.to_datetime(
    order1.order_approved_at, errors='coerce')
order1.order_delivered_carrier_date = pd.to_datetime(
    order1.order_delivered_carrier_date, errors='coerce')
order1.order_delivered_customer_date = pd.to_datetime(
    order1.order_delivered_customer_date, errors='coerce')
order1.order_estimated_delivery_date = pd.to_datetime(
    order1.order_estimated_delivery_date, errors='coerce')

##############################################################################
# difference de temps (c est vraimeent histoire 2)
order1['tim2approv'] = order1['order_approved_at'] - \
    order1['order_purchase_timestamp']
order1['tim2carrier'] = order1['order_delivered_carrier_date'] - \
    order1['order_approved_at']
order1['tim2deliv'] = order1['order_delivered_customer_date'] - \
    order1['order_delivered_carrier_date']
order1['tim2estim'] = order1['order_estimated_delivery_date'] - \
    order1['order_delivered_customer_date']
order1['tim'] = order1['order_delivered_customer_date'] - \
    order1['order_purchase_timestamp']

##############################################################################
# transfo des types timedelta en float
order1.tim2approv = order1.tim2approv.dt.total_seconds()/3600/24
order1.tim2carrier = order1.tim2carrier.dt.total_seconds()/3600/24
order1.tim2deliv = order1.tim2deliv.dt.total_seconds()/3600/24
order1.tim2estim = order1.tim2estim.dt.total_seconds()/3600/24
order1.tim = order1.tim.dt.total_seconds()/3600/24

In [ ]:
"""Jointure des tables olist: order,customer et order review."""
##############################################################################
# fusion order et customer
ordercust = order1.copy()
ordercust = ordercust.reset_index()
cust.columns = ['customer_id', 'customer_unique_id',
                'cust_zip', 'cust_city', 'cust_state']
ordercust1 = pd.merge(ordercust, cust, how='outer', on='customer_id')
##############################################################################
# on ne garde que les delivred
ordercust1 = ordercust1.loc[ordercust1.order_status == "delivered", :]
ordercust1 = ordercust1.drop(columns=['order_status',
                                      'order_approved_at',
                                      'order_delivered_carrier_date',
                                      'order_delivered_customer_date',
                                      'order_estimated_delivery_date'])

##############################################################################
# il peut y avoir plusieurs echanges pour les review : on garde la finale
def doublonrem(datain, datanam, keynam):
    temp = datain.copy()
    temp2 = temp.drop_duplicates(subset=keynam, keep='last', inplace=False)
    # inplace -> renvoie une table filtrée
    # ou modifie directement la table de base sans rien renvoyer
    print('extraction de ', temp2.shape[0]-datain.shape[0],
          'doublons de la table ', datanam, ' basée sur la variable ', keynam)
    return temp2

##############################################################################
# fusion order et order review
order_review1 = doublonrem(order_review, 'order_review', 'order_id')
# *le *1 permet de transformer le boolean en 0/1
order_review1['rev_title_flag'] =\
    order_review1.review_comment_title.notna()*1
order_review1['rev_comment_flag'] =\
    order_review1.review_comment_message.notna()*1

order_review1 = order_review1.drop(columns=['review_id',
                                            'review_comment_title',
                                            'review_comment_message',
                                            'review_creation_date',
                                            'review_answer_timestamp',
                                            'rev_title_flag'])
order_review1.columns = ['order_id', 'rev_score', 'rev_com_flag']

ordercust2 = ordercust1.set_index('order_id').join(
    order_review1.set_index('order_id'))

In [ ]:
"""
    Jointure des tables ordercust avec la table olist geo
    
    un peu de nettoyage sur les données geo
    aggregation des données par zip et état
    calcul de la distance entre zip et Sao Paulo (considéré comme centre)
    calcul de la distance entre etat et etat de Sao Paulo.
"""
##############################################################################
geo1 = geo.copy()
# petit remplissage pour empecher les trous de seller geo
geoapp = pd.DataFrame([[2285, -23.54, -46.63, 'sao paulo', 'SP'],
                       [7412, -23.3916478, -46.38547, 'aruja', 'SP'],
                       [37708, -21.8199612, -46.825645,
                        'pocos de caldas', 'MG'],
                       [71551, -15.7215826, -48.2182329, 'brasilia', 'DF'],
                       [72580, -15.7215826, -48.2182329, 'brasilia', 'DF'],
                       [82040, -25.4950497, -49.4302258, 'curitiba', 'PR'],
                       [91901, -30.1087954, -51.3175691, 'porto alegre',
                        'RS']], columns=['geolocation_zip_code_prefix',
                                         'geolocation_lat', 'geolocation_lng',
                                         'geolocation_city',
                                         'geolocation_state'])

geo1 = geo1.append(geoapp)
##############################################################################
# nettoyage
# on enleve les positions hors du bresil aberrantes
# et on fixe le sproblemes d'orthographe
geo1.loc[((geo1.geolocation_lng > -28) | (geo1.geolocation_lng < -75) |
          (geo1.geolocation_lat > 5)
          | (geo1.geolocation_lat < -34)), ['geolocation_lat',
                                            'geolocation_lng']] = np.nan
# minuscule
geo1['geolocation_city'] = geo1['geolocation_city'].str.lower()
# accent
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace('ã', 'a')
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace('í', 'i')
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace('é', 'e')
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace('ê', 'e')
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace('ô', 'o')
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace('á', 'a')
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace('ú', 'u')
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace('â', 'a')
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace('ó', 'o')
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace("t'", 't')
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace('õ', 'o')
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace('ç', 'c')
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace("d'", "d ")
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace(
    "embu", 'embu das artes')
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace(
    "das artes das artes", "das artes")
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace("-", ' ')

geo1['geolocation_city'] = geo1['geolocation_city'].str.replace(
    "piumhi", 'piumhii')
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace(
    "santa terezinha", 'santa teresinha')
geo1['geolocation_city'] = geo1['geolocation_city'].str.replace(
    "belem de sao francisco", 'belem do sao francisco')

##############################################################################
# aggregation des données geo par zip
geo1_a = geo1.groupby(by="geolocation_zip_code_prefix").agg(
    {'geolocation_lat': ['median'], 'geolocation_lng': ['median']})
geo1_a.columns = geo1_a.columns.droplevel(0)
geo1_a.columns = ['lat_zip', 'lng_zip']

geo1 = doublonrem(geo1, 'geo', 'geolocation_zip_code_prefix')
geo2 = geo1.loc[:, ['geolocation_zip_code_prefix', 'geolocation_city',
                    'geolocation_state']].set_index(
    'geolocation_zip_code_prefix')
geo2 = geo2.join(geo1_a)

##############################################################################
geo1_b = geo2.groupby(by="geolocation_state").agg(
    {'lat_zip': ['median'], 'lng_zip': ['median']})
geo1_b.columns = geo1_b.columns.droplevel(0)
geo1_b.columns = ['lat_state', 'lng_state']

geo2 = geo2.reset_index()
geo2 = geo2.set_index('geolocation_state')

geo2 = geo2.join(geo1_b)
geo2 = geo2.reset_index()

##############################################################################
# calcul distance depuis sao paulo (qu on qualifie de centre)
sao = geo2.loc[geo2.geolocation_city == 'sao paulo', :].groupby(
    by="geolocation_city").agg({'lat_zip': ['median'], 'lng_zip': ['median']})
sao.columns = sao.columns.droplevel(0)
sao.columns = ['sao_lat', 'sao_lng']
geo2['sao_lat'] = sao.sao_lat[0]
geo2['sao_lng'] = sao.sao_lng[0]

def distance(s_lat, s_lng, e_lat, e_lng):
    """calul de la distance par la formule de haversine.
    
    prend en entrée la latitude et longitude du point A puis du point B
    """

    # approximate radius of earth in km
    R = 6373.0

    s_lat = s_lat*np.pi/180.0
    s_lng = np.deg2rad(s_lng)
    e_lat = np.deg2rad(e_lat)
    e_lng = np.deg2rad(e_lng)

    d = np.sin((e_lat - s_lat)/2)**2 + np.cos(s_lat) * \
        np.cos(e_lat) * np.sin((e_lng - s_lng)/2)**2

    return 2 * R * np.arcsin(np.sqrt(d))


geo2['sdist_zip'] = distance(
    geo2.lat_zip, geo2.lng_zip, geo2.sao_lat, geo2.sao_lng)
geo2['sdist_state'] = distance(
    geo2.lat_state, geo2.lng_state, geo2.sao_lat, geo2.sao_lng)
geo2['sdist_dif'] = geo2['sdist_zip']-geo2['sdist_state']
# filtre coordonnées fausses car ne semble pas dans le bon etat :
# coordonnées de zip trop lointaines
# parfois dues à la presence de 2 villes differentes mais de meme nom

geo2.loc[(geo2['sdist_dif'].abs() > 600), ['lat_zip', 'lng_zip']] = np.nan
geo2.loc[geo2.lat_zip.isna(), ['lat_zip']] = geo2.lat_state
geo2.loc[geo2.lat_zip.isna(), ['lng_zip']] = geo2.lng_state
geo2['sdist_zip'] = distance(
    geo2.lat_zip, geo2.lng_zip, geo2.sao_lat, geo2.sao_lng)
geo2 = geo2.drop(columns='sdist_dif')

##############################################################################
ordercust2 = ordercust2.reset_index()
ordercust2 = ordercust2.set_index('cust_zip')

# minuscule
ordercust2['cust_city'] = ordercust2['cust_city'].str.lower()

ordercust2['cust_city'] = ordercust2['cust_city'].str.replace('ã', 'a')
ordercust2['cust_city'] = ordercust2['cust_city'].str.replace('í', 'i')
ordercust2['cust_city'] = ordercust2['cust_city'].str.replace('é', 'e')
ordercust2['cust_city'] = ordercust2['cust_city'].str.replace('ê', 'e')
ordercust2['cust_city'] = ordercust2['cust_city'].str.replace('ô', 'o')
ordercust2['cust_city'] = ordercust2['cust_city'].str.replace('á', 'a')
ordercust2['cust_city'] = ordercust2['cust_city'].str.replace('ú', 'u')
ordercust2['cust_city'] = ordercust2['cust_city'].str.replace('â', 'a')
ordercust2['cust_city'] = ordercust2['cust_city'].str.replace('ó', 'o')
ordercust2['cust_city'] = ordercust2['cust_city'].str.replace("t'", 't')
ordercust2['cust_city'] = ordercust2['cust_city'].str.replace('õ', 'o')
ordercust2['cust_city'] = ordercust2['cust_city'].str.replace('ç', 'c')
ordercust2['cust_city'] = ordercust2['cust_city'].str.replace("d'", "d ")
ordercust2['cust_city'] = ordercust2['cust_city'].str.replace(
    "embu", 'embu das artes')
ordercust2['cust_city'] = ordercust2['cust_city'].str.replace(
    "das artes das artes", "das artes")
ordercust2['cust_city'] = ordercust2['cust_city'].str.replace("-", ' ')

ordercustgeo = ordercust2.join(geo2.set_index('geolocation_zip_code_prefix'))

# Jointure finale

In [ ]:
"""
    Jointure des tables product3 avec la table olist geo
    puis jointure avec ordercustgeo
    pour memoire : product3 : jointure product, product cat, seller
                    ordercustgeo : order,customer, order review et geo
                    
    calcul distance seller-customer   
    aggregation de données
"""

##############################################################################
product4 = product3.reset_index()
product4 = product4.set_index('sel_zip')
geo3 = geo2.set_index('geolocation_zip_code_prefix')
productgeo = product4.join(geo3)


productgeo.columns = ['product_id', 'seller_id', 'order_id', 'order_item_id',
                      'ord_item_ship_lim_dat', 'ord_item_price',
                      'ord_item_fdp', 'sel_city',
                      'sel_state_', 'prod_desc_l', 'prod_foto_nb',
                      'prod_w', 'prod_cat',
                      'prod_vol', 'sel_state', 'sel_city_2', 'sel_lat_zip',
                      'sel_lng_zip', 'sel_lat_state', 'sel_lng_state',
                      'sao_lat', 'sao_lng', 'sel_sdist_zip',
                      'sel_sdist_state']
productgeo = productgeo.drop(columns='sel_state_')
# on doit faire confiance a sel_state_2 (plus fiable) issu de geo ,
# plutot que sel_state

##############################################################################
# lien entre les 2: order_id
ordercustgeo_light = ordercustgeo.copy()
ordercustgeo_light = ordercustgeo_light.reset_index()
ordercustgeo_light = ordercustgeo_light[[
    'order_id', 'lat_zip', 'lng_zip', 'lat_state', 'lng_state']]
ordercustgeo_light = ordercustgeo_light.set_index('order_id')

productgeo = productgeo.reset_index()
productgeo = productgeo.set_index('order_id')

productgeo2 = productgeo.join(ordercustgeo_light)

##############################################################################
# distance seller-customer
productgeo2['selcustdist_zip'] = distance(
    productgeo2.lat_zip, productgeo2.lng_zip, productgeo2.sel_lat_zip,
    productgeo2.sel_lng_zip)
productgeo2['selcustdist_state'] = distance(
    productgeo2.lat_state, productgeo2.lng_state, productgeo2.sel_lat_state,
    productgeo2.sel_lng_state)

listodr1 = ['index', 'sel_city', 'sel_state', 'sel_city_2',
            'sel_lat_zip', 'sel_lng_zip',
            'sel_lat_state', 'sel_lng_state', 'sao_lat', 'sao_lng',
            'lat_zip', 'lng_zip', 'lat_state', 'lng_state']
productgeo3 = productgeo2.drop(columns=listodr1)

listodr2 = ['cust_city', 'cust_state', 'geolocation_state',
            'geolocation_city', 'lat_zip',
            'lng_zip', 'lat_state', 'lng_state', 'sao_lat', 'sao_lng']
ordercustgeo = ordercustgeo.drop(columns=listodr2)

##############################################################################
# agg par order de productgeo3
productgeo3 = productgeo3.reset_index()
productgeo_agg = productgeo3.groupby(by="order_id").agg({
    'order_item_id': ['count', 'max'],
    'ord_item_price': ['sum'],
    'ord_item_fdp': ['sum'],
    'prod_desc_l': ['mean'],
    'prod_foto_nb': ['mean'],
    'prod_w': ['sum'],
    'prod_vol': ['sum'],
    'sel_sdist_zip': ['sum', 'mean'],
    'sel_sdist_state': ['sum', 'mean'],
    'selcustdist_zip': ['sum', 'mean'],
    'selcustdist_state': ['sum', 'mean']})
productgeo_agg.columns = productgeo_agg.columns.droplevel(0)
productgeo_agg.columns = ['item_nb', 'item_max', 'price', 'fdp',
                          'desc_l', 'foto', 'w', 'vol',
                          'sel2s_zip_sum', 'sel2s_zip_mean',
                          'sel2s_state_sum', 'sel2s_state_mean',
                          'sel2c_zip_sum', 'sel2c_zip_mean',
                          'sel2c_state_sum', 'sel2c_state_mean']

ordercustgeo2 = ordercustgeo.copy()
ordercustgeo2 = ordercustgeo2.set_index('order_id')
ordercustgeo2 = ordercustgeo2.join(productgeo_agg)


##############################################################################
# il faut trouver la categorie la plus achetee par commande
# utiliser un get dummies pour facilite le calcul

pd.set_option('max_columns', None)
display(ordercustgeo2)
display(productgeo3)
pd.set_option('max_columns', 10)

# recherche de la categorie majoritaire en CA dans la commande

# one hot encoder
productgeo4 = pd.get_dummies(data=productgeo3, columns=['prod_cat'])
liscat = productgeo4.filter(regex='^prod_cat_', axis=1).columns.tolist()
liscat2 = [s.replace("prod_cat_", "") for s in liscat]
# display(liscat2)

productgeo5 = productgeo4[['order_id', 'ord_item_price']+liscat]
productgeo6 = pd.concat([productgeo5['order_id'], productgeo5.filter(
    regex='^prod_cat_', axis=1).multiply(productgeo5['ord_item_price'],
                                         axis="index")], axis=1)
# aggregation
productgeo6_agg = productgeo6.groupby(by="order_id").agg(['sum'])
productgeo6_agg.columns = productgeo6_agg.columns.droplevel(0)
productgeo6_agg.columns = liscat2
# categorie max
productgeo6_agg['cat'] = productgeo6_agg.idxmax(axis=1)

# fusion avec ordercustgeo2
ordercustgeo3 = ordercustgeo2.join(productgeo6_agg['cat'])

ordercustgeo3 = ordercustgeo3[['customer_id', 'customer_unique_id',
                               'order_purchase_timestamp',
                               'tim2approv', 'tim2carrier',
                               'tim2deliv', 'tim2estim', 'tim', 'item_nb',
                               'item_max', 'price', 'fdp', 'w', 'vol', 'cat',
                               'pay_seq', 'pay_ins', 'pay_val',
                               'pay_val_voucher', 'pay_val_pct_voucher',
                               'pay_type',
                               'rev_score', 'rev_com_flag', 'desc_l',
                               'foto', 'sdist_zip', 'sdist_state',
                               'sel2s_zip_sum',
                               'sel2s_zip_mean', 'sel2s_state_sum',
                               'sel2s_state_mean', 'sel2c_zip_sum',
                               'sel2c_zip_mean',
                               'sel2c_state_sum', 'sel2c_state_mean']]
ordercustgeo3 = ordercustgeo3.sort_values('order_purchase_timestamp')
orderfull = ordercustgeo3.copy()

catfull = productgeo3[['order_id', 'order_item_id', 'ord_item_price',
                       'ord_item_fdp',
                       'prod_desc_l', 'prod_foto_nb', 'prod_w', 'prod_cat',
                       'prod_vol']].copy()
catfull = catfull.set_index('order_id')
catfull = catfull.join(orderfull['order_purchase_timestamp'])
catfull = catfull.loc[catfull.order_purchase_timestamp.notna(),
                      :].sort_values(
    ['order_purchase_timestamp', 'order_item_id'])

# Check données manquantes et imputer

In [ ]:
"""
    Check des données manquantes
"""
orderfull = orderfull.loc[orderfull.pay_seq.notna(), :]
orderfull = orderfull.loc[orderfull.pay_type.notna(), :]
orderfull['rev_score'] = orderfull['rev_score'].fillna(2.5)
orderfull['rev_com_flag'] = orderfull['rev_com_flag'].fillna(0)
orderfull['desc_l'] = orderfull['desc_l'].fillna(0)
orderfull['foto'] = orderfull['foto'].fillna(0)

orderfull['temp'] = orderfull.tim-orderfull.tim2deliv-1
orderfull.loc[orderfull.tim2carrier.isna(), ['tim2carrier']
              ] = orderfull['temp']
orderfull.loc[orderfull.tim2approv.isna(), ['tim2approv']] = 1

orderfull.loc[(orderfull.sel2c_zip_mean.isna() &
               orderfull.sel2c_zip_sum == 0), 'sel2c_zip_mean'] = 0
orderfull.loc[(orderfull.sel2c_state_mean.isna() &
               orderfull.sel2c_state_sum == 0), 'sel2c_state_mean'] = 0

orderfull.loc[(orderfull.sel2s_zip_mean.isna() &
               orderfull.sel2s_zip_sum == 0), 'sel2c_zip_mean'] = 0
orderfull.loc[(orderfull.sel2s_state_mean.isna() &
               orderfull.sel2s_state_sum == 0), 'sel2c_state_mean'] = 0

orderfull = orderfull.drop(columns=['temp'])

catfull = catfull.loc[catfull.prod_cat.notna()]
catfull = catfull.loc[catfull.prod_w.notna()]

plt.rcParams["figure.figsize"] = [16, 6]
plt.rcParams["figure.autolayout"] = True
f, axes = plt.subplots(1, 2)

ordernull = 1-(orderfull.isnull().sum()/orderfull.shape[0])
ordernull.plot.barh(color="#8d19a9", title='Pourcentage de données',
                    xlabel='pourcentage de données', ax=axes[0])
axes[0].grid(zorder=0)

catnull = 1-(catfull.isnull().sum()/catfull.shape[0])
catnull.plot.barh(color="#8d19a9", title='Pourcentage de données',
                  xlabel='pourcentage de données', ax=axes[1])
axes[1].grid(zorder=0)

plt.show()

pd.set_option('max_rows', None)
display(ordernull)
display(catnull)
pd.set_option('max_rows', 10)

In [ ]:
# KNN imputer
from sklearn.impute import KNNImputer
import timeit

start = timeit.default_timer()

imputer = KNNImputer(n_neighbors=5)

orderfull2 = orderfull.copy()
orderfull2 = orderfull2.reset_index()
orderfull3 = orderfull2.drop(columns=[
                             'order_id', 'customer_id', 'customer_unique_id',
                             'order_purchase_timestamp', 'cat', 'pay_type'])

# wesh=orderfull2.head(1000)
imputer.fit(orderfull3)
orderfull4 = pd.DataFrame(imputer.transform(
    orderfull3), columns=orderfull3.columns)

stop1 = timeit.default_timer()
print('Time: ', stop1 - start)

orderfull = pd.concat([orderfull2[['order_id', 'customer_id',
                                   'customer_unique_id',
                                   'order_purchase_timestamp', 'cat',
                                   'pay_type']], orderfull4], axis=1)

In [ ]:
pd.set_option('max_rows', None)
display(orderfull.dtypes)
pd.set_option('max_rows', 10)

# Fonction IQR filter et Correlation

In [ ]:
# fonction IQR filter
# filtre IQR car les minmax ou standard scaler sont sensibles aux outliers
from scipy.stats import normaltest
from scipy.stats import shapiro
from scipy.stats import spearmanr
from scipy.stats import pearsonr


def iqr_outlier(df, coef=1.5, reset=True, cutupQ=True):
    """Fonction qui traite les outliers
    
        dans notre cas ne traite que les données positives
    
        prend en entrée le dataframe
        reset=true: reset index
        cutupQ=True: cap à q99
        coef=1.5 coef de l'iqr pour capper et floorer
    """
    if reset == True:
        df = df.reset_index()
    df1 = df.copy()

    df = df._get_numeric_data()

    pd.set_option('max_columns', None)
    # display(df)
    pd.set_option('max_columns', 10)

    q1 = df.quantile(0.25)
    q3 = df.quantile(0.75)
    q99 = df.quantile(0.99)
    print('q1')
    pd.set_option('max_rows', None)
    display(q1)
    pd.set_option('max_rows', 10)
    print('q3')
    pd.set_option('max_rows', None)
    display(q3)
    pd.set_option('max_rows', 10)

    iqr = q3 - q1

    print('IQR')
    pd.set_option('max_rows', None)
    display(iqr)
    pd.set_option('max_rows', 10)

    print('Q99')
    pd.set_option('max_rows', None)
    display(q99)
    pd.set_option('max_rows', 10)

    lower_bound = q1 - (coef * iqr)
    upper_bound = q3 + (coef * iqr)

    for col in df.columns:
        for i in range(0, df.shape[0]):
            # evitons les inferieurs à 0 dans notre cas
            if (cutupQ == False):
                if df.iloc[i, df.columns.get_loc(col)] < lower_bound[col]:
                    df.iloc[i, df.columns.get_loc(col)] = max(
                        0, lower_bound[col])

                if df.iloc[i, df.columns.get_loc(col)] > upper_bound[col]:
                    df.iloc[i, df.columns.get_loc(col)] = upper_bound[col]

            if (cutupQ == True):
                if df.iloc[i, df.columns.get_loc(col)] > q99[col]:
                    df.iloc[i, df.columns.get_loc(col)] = q99[col]

    # display(df1)
    # display(df)
    for col in df.columns:
        df1.loc[:, col] = df.loc[:, col]

    if reset == True:
        return(df1.set_index(df1.columns[0]))
    else:
        return(df1)


def calculate_corr(df):
    """Fonction qui calcul des correlations."""
    
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pearscoeff = dfcols.transpose().join(dfcols, how='outer')
    pearspvalues = pearscoeff.copy()
    shappvalues = pearscoeff.copy()
    normpvalues = pearscoeff.copy()
    spearcoeff = pearscoeff.copy()
    spearpvalues = pearscoeff.copy()
    for r in df.columns:
        for c in df.columns:
            pearscoeff[r][c] = round(pearsonr(df[r], df[c])[0], 4)
            pearspvalues[r][c] = round(pearsonr(df[r], df[c])[1], 4)
            spearcoeff[r][c] = round(spearmanr(df[r], df[c])[0], 4)
            spearpvalues[r][c] = round(spearmanr(df[r], df[c])[1], 4)
            if r == c:
                # attention pvalue shapiro wilk seulement pour N<5000
                shappvalues[r][c] = round(shapiro(df[r])[1], 4)
                # d'agostino basé sur omnibus
                # plus adapté pour les grands echantillons
                normpvalues[r][c] = round(normaltest(df[r])[1], 4)
    pearscoeff[df.columns] = pearscoeff[df.columns].astype(float)
    pearspvalues[df.columns] = pearspvalues[df.columns].astype(float)
    shappvalues[df.columns] = shappvalues[df.columns].astype(float)
    normpvalues[df.columns] = normpvalues[df.columns].astype(float)
    spearcoeff[df.columns] = spearcoeff[df.columns].astype(float)
    spearpvalues[df.columns] = spearpvalues[df.columns].astype(float)
    return pearscoeff, pearspvalues, shappvalues,\
normpvalues, spearcoeff, spearpvalues

# SEGMENTATION
* k-means algorithm is very fast
* but it falls in local minima. That’s why it can be useful to restart it several times
* If the algorithm stops before fully converging,the cluster_centers_ will not be the means of the points in each cluster


In [ ]:
def cutdata(cutdate='2017-10-01', correl=False):
    """Fonction qui prepare les données avant segmentation.
    
        selection de la periode via cutdate
        ajout des données de catégories par features
        nombre de commande LTM
        recence
        affichage des correlations si besoin
    """
    
    
    

    # les categories
    # selection date
    cat2seg = catfull.loc[(catfull['order_purchase_timestamp'] < cutdate)]

    # aggregation par categorie
    cat2seg_agg = cat2seg.groupby(by="prod_cat").agg({
        'prod_vol': ['median'],
        'prod_w': ['median'],
        'order_item_id': ['count'],
        'ord_item_price': ['sum', 'median'],
        'ord_item_fdp': ['sum', 'median'],
        'prod_desc_l': ['median'],
        'prod_foto_nb': ['median']})
    cat2seg_agg.columns = cat2seg_agg.columns.droplevel(0)
    cat2seg_agg.columns = ['cat_vol', 'cat_w', 'cat_nb', 'cat_ca',
                           'cat_pr', 'cat_fdp_ca',
                           'cat_fdp', 'cat_des', 'cat_fot']

    cat2seg2 = cat2seg_agg
    # on reduit le nombre de variables
    # en regardant les correlations precedentes
    cat2seg2b = cat2seg2.drop(
        columns=['cat_vol', 'cat_fdp', 'cat_nb', 'cat_fdp_ca', 'cat_des'])

    #################################
    #################################
    #################################
    if correl == True:
        cocorr, pvcorr, shap, dagos, cospcorr, pvspcorr = calculate_corr(
            cat2seg2)

        plt.rcParams["figure.figsize"] = [16, 6]
        f, axes = plt.subplots(1, 2)

        sns.heatmap(
            cocorr,
            vmin=-1, vmax=1, center=0,
            cmap=sns.diverging_palette(20, 220, n=200),
            square=True,
            annot=True,
            fmt='.2f',
            linewidths=.5, ax=axes[0])
        axes[0].title.set_text('Coefficient correlation de Pearson')

        sns.heatmap(
            pvcorr,
            vmin=0, vmax=1, center=0.5,
            cmap=sns.diverging_palette(200, 20, n=200),
            square=True,
            annot=True,
            fmt='.2f',
            linewidths=.5, ax=axes[1])
        axes[1].title.set_text('Pvalue correlation de Pearson')

        plt.show()

        ########################################

        cocorr, pvcorr, shap, dagos, cospcorr, pvspcorr = calculate_corr(
            cat2seg2b)

        plt.rcParams["figure.figsize"] = [16, 6]
        f, axes = plt.subplots(1, 2)

        sns.heatmap(
            cocorr,
            vmin=-1, vmax=1, center=0,
            cmap=sns.diverging_palette(20, 220, n=200),
            square=True,
            annot=True,
            fmt='.2f',
            linewidths=.5, ax=axes[0])
        axes[0].title.set_text('Coefficient correlation de Pearson')

        sns.heatmap(
            pvcorr,
            vmin=0, vmax=1, center=0.5,
            cmap=sns.diverging_palette(200, 20, n=200),
            square=True,
            annot=True,
            fmt='.2f',
            linewidths=.5, ax=axes[1])
        axes[1].title.set_text('Pvalue correlation de Pearson')

        plt.show()
    #################################
    #################################
    #################################

    # frequence de commande last 12 months
    # table last 12 months pour la frequ
    freq = orderfull.loc[((orderfull['order_purchase_timestamp'] < cutdate) & (
        orderfull['order_purchase_timestamp'] >
        pd.to_datetime(cutdate)-pd.DateOffset(years=1))),
        ['customer_unique_id', 'item_nb']]

    # calcul frequence de commande last 12 months
    freq_agg = freq.groupby(by="customer_unique_id").agg(
        {'item_nb': ['count']})
    freq_agg.columns = freq_agg.columns.droplevel(0)
    freq_agg.columns = ['nb_com_ltm']

    # le gros des données
    # selection date
    order2seg = orderfull.loc[(
        orderfull['order_purchase_timestamp'] < cutdate), :].copy()
    order2seg['init'] = dt.datetime.strptime(cutdate, '%Y-%m-%d')
    order2seg['rec'] = (order2seg['order_purchase_timestamp'] -
                        order2seg['init']).dt.total_seconds()/3600/24

    order2segb = order2seg.reset_index()
    # jointure avec les categories
    order2segb = order2segb.set_index('cat').join(cat2seg2b)
    order2segb = order2segb.reset_index()
    order2segb = order2segb.sort_values('index')
    order2segb = order2segb.set_index('index')

    # aggregation sur client unique
    # item_nb : nombre d 'article' -> count= nombre de commandes
    # last = nombre d'objet de la derniere commande
    # sum = nombre d'objet commandés
    # prix : prix commande sans fdp' -> #sum = somme totale depensée
    # last = prix derniere commande
    # mean = prix moyen par commande
    # fdp : prix commande sans fdp' -> #sum = fdp totaux depensés
    # last = fdp derniere commande
    # mean = fdp moyen par commande
    # w : poids'                    -> #sum = poids de toutes les commandes
    # last = poids derniere commande
    # mean = poids moyen par commande
    # vol : volume'        -> #sum = volume total de toutes les commandes
    # last = volume derniere commande
    # mean = volume moyen par commande
    # pay_inst : paiements credit ' -> #sum = nombre total de paiements
    # last = nombre de paiements pour la derniere commande
    # mean = nombre de paiements moyen par commande
    # rev_score : satisfaction ' -> #last = dernier score
    # mean = score moyen
    # rev_comm_flag : commentaire ' -> #last = derniere commande
    #mean = moyenne
    # rsdist_zip : distance zip ' -> #last = derniere commande
    # sdist_state : distance etat ' -> #last = derniere commande

    # cat_w: poids categorie  ->#last ->
    # poids categorie de la derniere commande
    # cat_ca: poids categorie  ->#last ->
    # ca moyen categorie de la derniere commande
    # cat_pr: poids categorie  ->#last ->
    # prix moyen article categorie de la derniere commande
    # cat_fot: poids categorie  ->#last ->
    # nb photos categorie de la derniere commande

    # recence -> #last derniere commande
    order2seg_agg = order2segb.groupby(by="customer_unique_id").agg({
        'item_nb': ['count', 'last', 'sum'],
        'price': ['sum', 'last', 'mean'],
        'fdp': ['sum', 'last', 'mean'],
        'w': ['sum', 'last', 'mean'],
        'vol': ['sum', 'last', 'mean'],
        'pay_ins': ['sum', 'last', 'mean'],
        'pay_val_pct_voucher': ['mean'],
        'rev_score': ['last', 'mean'],
        'rev_com_flag': ['last', 'mean'],
        'sdist_zip': ['last'],
        'sdist_state': ['last'],
        'cat_w': ['last'],
        'cat_ca': ['last'],
        'cat_pr': ['last'],
        'cat_fot': ['last'],
        'rec': ['last']})
    order2seg_agg.columns = order2seg_agg.columns.droplevel(0)
    order2seg_agg.columns = ['nb_com', 'item_nb_last_com', 'item_nb_total',
                             'pr_sum', 'pr_last', 'pr_mean',
                             'fdp_sum', 'fdp_last', 'fdp_mean',
                             'w_sum', 'w_last', 'w_mean',
                             'vol_sum', 'vol_last', 'vol_mean',
                             'pay_ins_sum', 'pay_ins_last', 'pay_ins_mean',
                             'pct_voucher', 'note_last', 'note_mean',
                             'com_last', 'com_mean',
                             'dist_zip', 'dist_state',
                             'cat_w_last', 'cat_ca_last',
                             'cat_pr_last', 'cat_fot_last',
                             'rec']
    # jointure avec freq last 12m
    order2seg_agg = order2seg_agg.join(freq_agg)
    order2seg_agg['nb_com_ltm'] = order2seg_agg['nb_com_ltm'].fillna(0)

    #####################
    # correlation
    if correl == True:
        cocorr, pvcorr, shap, dagos, cospcorr, pvspcorr = calculate_corr(
            order2seg2)

        plt.rcParams["figure.figsize"] = [16, 12]
        f, axes = plt.subplots(1, 1)

        sns.heatmap(
            cocorr,
            vmin=-1, vmax=1, center=0,
            cmap=sns.diverging_palette(20, 220, n=200),
            square=True,
            annot=True,
            annot_kws={"size": 50 / np.sqrt(len(cocorr))},
            fmt='.2f',
            linewidths=.5, ax=axes)
        axes.title.set_text('Coefficient correlation de Pearson')
    return(order2seg_agg)

In [ ]:
# kmeans (yellowbrick- machine learning vizualization)
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import scipy.stats as st
import warnings
import datetime as dt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer,\
InterclusterDistance


def seg_kmeans(
        cutD='2017-10-01',
        scalerfitD='2017-10-01',
        cutO=False,
        Standardization=True,
        scal='Standard',
        lisfac=['rec', 'nb_com_ltm', 'pr_mean'],
        ElbowVisualizer=True,
        Sil_ICD_Visualizer=True,
        Sil_ICD_K=[3, 4, 5, 6]):
    
    """Segmentation kmeans.
    
        cutD: selection de la periode
        scalerfitD: selection de la période de normalisation
        cutO: traitement des outliers ou non
        Standardization: normalisation ou non
        scal: type de normalisation 'Standard' ou 'minmax'
        lisfac: liste des features à selectionner
        ElbowVisualizer: Visu ou non des scores de distorsion, silhouette
        Sil_ICD_Visualizer: Visu ou non des clusters et de leurs caracteristiques
        Sil_ICD_K: liste des nombres de clusters à visualiser
    """

    # warnings.filterwarnings('ignore')

    # selection date pour segmentation
    data2s = cutdata(cutdate=cutD, correl=False)
    # selection date pour scaler fit
    data2scalerfit = cutdata(cutdate=scalerfitD, correl=False)

    if cutO == True:
        # filtre IQR / complexe à utiliser
        # car les clients à plusieurs commandes sont rares
        # mieux vaut faire un cut up sur Q99 ou plus: cutupQ
        data2s = iqr_outlier(data2s, coef=1.5, cutupQ=True)
        data2scalerfit = iqr_outlier(data2scalerfit, coef=1.5, cutupQ=True)

    # selection des facteurs
    data2s_selec = data2s.loc[:, lisfac].copy()
    data2scalerfit_selec = data2scalerfit.loc[:, lisfac].copy()

    # scaler init
    # pour le clustering Standard est souvent prefere
    if Standardization == True:
        if scal == 'MinMax':
            scaler = MinMaxScaler()
        if scal == 'Standard':
            scaler = StandardScaler()
        # scaler fit
        scaler.fit(data2scalerfit_selec)
        # scaler transform
        data2s2_selec = pd.DataFrame(scaler.transform(
            data2s_selec), columns=data2s_selec.columns,
            index=data2s_selec.index)
    else:
        data2s2_selec = data2s_selec

    # kmeans avec yellowbrick
    # implements the “elbow” method of selecting the optimal number of clusters
    # by fitting the K-Means model with a range of values for K
    # the “elbow” (the point of inflection on the curve) is a good indication
    # that the underlying model fits best at that point

    # the elbow method doesn't always work well; especially
    # if the data is not very clustered.
    # No clear elbow. Instead, we see a fairly smooth curve,
    # unclear what is the best value of k to choose

    # attention def legerment differente entre scikitlearn et yellowbrick
    # DISTORSION to minimize: sum of squared distances
    # from each point to its assigned center.
    # CALINSKI to minimize ,dispersion within clusters /
    # dispersion between clusters /
    # silhouette to minimize mean ratio of intra-cluster
    # and nearest-cluster distance
    # init
    if ElbowVisualizer == True:

        model = KMeans()

        plt.rcParams["figure.figsize"] = [16, 5]
        plt.rcParams["figure.autolayout"] = True
        f, axes = plt.subplots(1, 3)

        # distorsion
        visualizer1 = KElbowVisualizer(model, k=(3, 12), ax=axes[0])
        visualizer1.fit(data2s2_selec)    # Fit the data to the visualizer
        visualizer1.finalize()  # prepare the chart : title /label
        # CALINSKI
        visualizer2 = KElbowVisualizer(
            model, k=(3, 12), metric='calinski_harabasz', ax=axes[2])
        # , timings=False)
        visualizer2.fit(data2s2_selec)    # Fit the data to the visualizer
        visualizer2.finalize()  # prepare the chart : title /label
        # CALINSKI
        visualizer3 = KElbowVisualizer(
            model, k=(3, 12), metric='silhouette', ax=axes[1])
        # , timings=False)
        visualizer3.fit(data2s2_selec)    # Fit the data to the visualizer
        visualizer3.finalize()  # prepare the chart : title /label

        plt.show()

    # evaluate the density and separation between clusters
    # calculated by averaging the silhouette coefficient for each sample:
        # computed as the difference between the average intra-cluster
        # distance and the mean nearest-cluster distance for each sample
        # normalized by the maximum value

        # scores near +1 indicate high separation
        # scores near -1 indicate that the samples may have been
        # assigned to the wrong cluster.

    # The SilhouetteVisualizer displays the silhouette coefficient
    # for each sample on a per-cluster basis,
    # allowing users to visualize the density and separation of the clusters

    # Intercluster distance maps :cluster centers in 2 dimensions
    # with the distance to other centers preserved.
    # E.g. the closer to centers are in the visualization,
    # the closer they are in the original feature space.
    # The clusters are sized according to a scoring metric.
        # By default, they are sized by membership,
        # e.g. the number of instances that belong to each center.
        # This gives a sense of the relative importance of clusters.
    # Note however, that because two clusters overlap in the 2D space,
    # it does not imply that they overlap in the original
    plt.rcParams["figure.figsize"] = [16, 5]
    plt.rcParams["figure.autolayout"] = False
    if Sil_ICD_Visualizer == True:

        for i in Sil_ICD_K:
            f, axes = plt.subplots(1, 2)

            model = KMeans(i)
            model.fit(data2s2_selec.values)

            visualizer = SilhouetteVisualizer(
                model, ax=axes[0], is_fitted=True)
            # deja fité juste au dessus
            # Fit the data to the visualizer
            visualizer.fit(data2s2_selec.values)
            visualizer.finalize()
            # prepare the chart : title /label

            visualizer = InterclusterDistance(
                model, ax=axes[1], is_fitted=True)
            # deja fité juste au dessus
            # Fit the data to the visualizer
            visualizer.fit(data2s2_selec.values)
            visualizer.finalize()  # prepare the chart : title /label

            plt.show()

            model.fit(data2s2_selec)
            kmeans_labels = model.labels_
            kmeans_centers = pd.DataFrame(
                model.cluster_centers_,
                columns=model.feature_names_in_.tolist())

            # display(model.labels_)
            # display(scaler.inverse_transform(data2s2_selec))

            # display(pd.value_counts(kmeans_labels,normalize=True))
            # display(kmeans_centers)

            # on percentilise le zcore #st.norm.ppf(.95) #st.norm.cdf(1.64)
            kmeans_centers = kmeans_centers.applymap(st.norm.cdf)
            # display(kmeans_centers)

            h = kmeans_centers.T
            h = h.reset_index()

            f, axes = plt.subplots(1, 2)

            axes[0].pie(pd.value_counts(kmeans_labels),
                        labels=pd.value_counts(
                kmeans_labels).index, normalize=True, autopct='%1.1f%%',
                shadow=True, startangle=90)
            # Equal aspect ratio ensures that pie is drawn as a circle.
            axes[0].axis('equal')
            axes[0].set_title('distribution des clusters')

            h.plot(x="index", y=kmeans_centers.index, kind="bar",
                   fontsize=12, width=0.7, ax=axes[1])
            axes[1].set_title('features des centroides des clusters')

            plt.show()

            # boxplot des clusters
            # on aurait pu prendre la table avant inversion directement
            temp1 = pd.concat([pd.DataFrame(
                scaler.inverse_transform(data2s2_selec),
                columns=lisfac, index=data2s2_selec.index),
                pd.DataFrame(model.labels_,
                             columns=['clus'],
                             index=data2s2_selec.index)],
                axis=1)
            temp2 = pd.concat([data2s2_selec,
                               pd.DataFrame(model.labels_,
                                            columns=['clus'],
                                            index=data2s2_selec.index)],
                              axis=1)

            plt.rcParams["figure.figsize"] = [16, 5]
            plt.rcParams["figure.autolayout"] = True

            # non standardisé
            # attention à ne pas mettre trop de facteurs
            f, axes = plt.subplots(1, len(lisfac))
            counter = 0
            for i in lisfac:
                sns.boxplot(x='clus', y=i, data=temp1, showmeans=True,
                            meanprops={
                                "marker": "s", "markerfacecolor": "pink",
                                "markeredgecolor": "blue"},
                            showfliers=False, ax=axes[counter])
                axes[counter].set_title(
                    'boxplot feature par cluster', fontsize=15)
                counter = counter+1

            plt.show()

            # plt.show()

            # scatterplot des clusters
            if len(lisfac) == 3:
                f, axes = plt.subplots(1, 3)
                sns.scatterplot(
                    data=temp1,
                    x=lisfac[0],
                    y=lisfac[1],
                    hue='clus',
                    ax=axes[0])
                sns.scatterplot(
                    data=temp1,
                    x=lisfac[0],
                    y=lisfac[2],
                    hue='clus',
                    ax=axes[1])
                sns.scatterplot(
                    data=temp1,
                    x=lisfac[1],
                    y=lisfac[2],
                    hue='clus',
                    ax=axes[2])
                axes[0].set_title(
                    'scatterplot feature par cluster', fontsize=15)
                axes[1].set_title(
                    'scatterplot feature par cluster', fontsize=15)
                axes[2].set_title(
                    'scatterplot feature par cluster', fontsize=15)
                plt.show()

                # 3D scatter
                dat3d = [go.Scatter3d(x=temp1[lisfac[0]],
                                      y=temp1[lisfac[1]],
                                      z=temp1[lisfac[2]],
                                      mode='markers',
                                      marker=dict(size=2,
                                                  color=temp1['clus'],
                                                  colorbar=dict(title="cluster")))]
                fig = go.Figure(dat3d)

                fig.update_layout(
                    title="cluster-scatter",
                    scene=dict(
                        xaxis_title=lisfac[0],
                        yaxis_title=lisfac[1],
                        zaxis_title=lisfac[2]),)

                fig.show()

            if len(lisfac) == 4:
                f, axes = plt.subplots(1, 3)
                sns.scatterplot(
                    data=temp1, x=lisfac[0], y=lisfac[1],
                    hue='clus', ax=axes[0])
                sns.scatterplot(
                    data=temp1, x=lisfac[0], y=lisfac[2],
                    hue='clus', ax=axes[1])
                sns.scatterplot(
                    data=temp1, x=lisfac[0], y=lisfac[3],
                    hue='clus', ax=axes[2])
                axes[0].set_title(
                    'scatterplot feature par cluster', fontsize=15)
                axes[1].set_title(
                    'scatterplot feature par cluster', fontsize=15)
                axes[2].set_title(
                    'scatterplot feature par cluster', fontsize=15)
                plt.show()

                f, axes = plt.subplots(1, 3)
                sns.scatterplot(
                    data=temp1, x=lisfac[1], y=lisfac[2],
                    hue='clus', ax=axes[0])
                sns.scatterplot(
                    data=temp1, x=lisfac[1], y=lisfac[3],
                    hue='clus', ax=axes[1])
                sns.scatterplot(
                    data=temp1, x=lisfac[2], y=lisfac[3],
                    hue='clus', ax=axes[2])
                axes[0].set_title(
                    'scatterplot feature par cluster', fontsize=15)
                axes[1].set_title(
                    'scatterplot feature par cluster', fontsize=15)
                axes[2].set_title(
                    'scatterplot feature par cluster', fontsize=15)
                plt.show()

                # 3D scatter
                dat3d = [go.Scatter3d(x=temp1[lisfac[0]],
                                      y=temp1[lisfac[1]], z=temp1[lisfac[2]],
                                      mode='markers',
                                      marker=dict(size=2,
                                                  color=temp1['clus'],
                                                  colorbar=dict(title="cluster")))]
                fig = go.Figure(dat3d)

                fig.update_layout(
                    title="cluster-scatter",
                    scene=dict(
                        xaxis_title=lisfac[0],
                        yaxis_title=lisfac[1],
                        zaxis_title=lisfac[2]),)

                fig.show()

                dat3d = [go.Scatter3d(x=temp1[lisfac[0]],
                                      y=temp1[lisfac[1]],
                                      z=temp1[lisfac[3]],
                                      mode='markers',
                                      marker=dict(size=2, color=temp1['clus'],
                                                  colorbar=dict(title="cluster")))]
                fig = go.Figure(dat3d)

                fig.update_layout(
                    title="cluster-scatter",
                    scene=dict(
                        xaxis_title=lisfac[0],
                        yaxis_title=lisfac[1],
                        zaxis_title=lisfac[3]),)

                fig.show()

                dat3d = [go.Scatter3d(x=temp1[lisfac[0]],
                                      y=temp1[lisfac[2]],
                                      z=temp1[lisfac[3]],
                                      mode='markers',
                                      marker=dict(size=2,
                                                  color=temp1['clus'],
                                                  colorbar=dict(title="cluster")))]
                fig = go.Figure(dat3d)

                fig.update_layout(
                    title="cluster-scatter",
                    scene=dict(
                        xaxis_title=lisfac[0],
                        yaxis_title=lisfac[2],
                        zaxis_title=lisfac[3]),)

                fig.show()

                dat3d = [go.Scatter3d(x=temp1[lisfac[1]],
                                      y=temp1[lisfac[2]],
                                      z=temp1[lisfac[3]],
                                      mode='markers',
                                      marker=dict(size=2,
                                                  color=temp1['clus'],
                                                  colorbar=dict(title="cluster")))]
                fig = go.Figure(dat3d)

                fig.update_layout(
                    title="cluster-scatter",
                    scene=dict(
                        xaxis_title=lisfac[1],
                        yaxis_title=lisfac[2],
                        zaxis_title=lisfac[3]),)

                fig.show()

            if len(lisfac) == 5:
                f, axes = plt.subplots(1, 4)
                sns.scatterplot(
                    data=temp1, x=lisfac[0], y=lisfac[1],
                    hue='clus', ax=axes[0])
                sns.scatterplot(
                    data=temp1, x=lisfac[0], y=lisfac[2],
                    hue='clus', ax=axes[1])
                sns.scatterplot(
                    data=temp1, x=lisfac[0], y=lisfac[3],
                    hue='clus', ax=axes[2])
                sns.scatterplot(
                    data=temp1, x=lisfac[0], y=lisfac[4],
                    hue='clus', ax=axes[3])
                axes[0].set_title(
                    'scatterplot feature par cluster', fontsize=15)
                axes[1].set_title(
                    'scatterplot feature par cluster', fontsize=15)
                axes[2].set_title(
                    'scatterplot feature par cluster', fontsize=15)
                axes[3].set_title(
                    'scatterplot feature par cluster', fontsize=15)
                plt.show()

                f, axes = plt.subplots(1, 3)
                sns.scatterplot(
                    data=temp1, x=lisfac[1], y=lisfac[2],
                    hue='clus', ax=axes[0])
                sns.scatterplot(
                    data=temp1, x=lisfac[1], y=lisfac[3],
                    hue='clus', ax=axes[1])
                sns.scatterplot(
                    data=temp1, x=lisfac[1], y=lisfac[4],
                    hue='clus', ax=axes[2])
                axes[0].set_title(
                    'scatterplot feature par cluster', fontsize=15)
                axes[1].set_title(
                    'scatterplot feature par cluster', fontsize=15)
                axes[2].set_title(
                    'scatterplot feature par cluster', fontsize=15)
                plt.show()

                f, axes = plt.subplots(1, 3)
                sns.scatterplot(
                    data=temp1, x=lisfac[2], y=lisfac[3],
                    hue='clus', ax=axes[0])
                sns.scatterplot(
                    data=temp1, x=lisfac[2], y=lisfac[4],
                    hue='clus', ax=axes[1])
                sns.scatterplot(
                    data=temp1, x=lisfac[3], y=lisfac[4],
                    hue='clus', ax=axes[2])
                axes[0].set_title(
                    'scatterplot feature par cluster', fontsize=15)
                axes[1].set_title(
                    'scatterplot feature par cluster', fontsize=15)
                axes[2].set_title(
                    'scatterplot feature par cluster', fontsize=15)
                plt.show()

                # 3D scatter
                dat3d = [go.Scatter3d(x=temp1[lisfac[0]],
                                      y=temp1[lisfac[1]],
                                      z=temp1[lisfac[2]],
                                      mode='markers',
                                      marker=dict(size=2,
                                                  color=temp1['clus'],
                                                  colorbar=dict(title="cluster")))]
                fig = go.Figure(dat3d)

                fig.update_layout(
                    title="cluster-scatter",
                    scene=dict(
                        xaxis_title=lisfac[0],
                        yaxis_title=lisfac[1],
                        zaxis_title=lisfac[2]),)

                fig.show()

                dat3d = [go.Scatter3d(x=temp1[lisfac[0]],
                                      y=temp1[lisfac[1]],
                                      z=temp1[lisfac[3]],
                                      mode='markers',
                                      marker=dict(size=2,
                                                  color=temp1['clus'],
                                                  colorbar=dict(title="cluster")))]
                fig = go.Figure(dat3d)

                fig.update_layout(
                    title="cluster-scatter",
                    scene=dict(
                        xaxis_title=lisfac[0],
                        yaxis_title=lisfac[1],
                        zaxis_title=lisfac[3]),)

                fig.show()

                dat3d = [go.Scatter3d(x=temp1[lisfac[0]],
                                      y=temp1[lisfac[1]],
                                      z=temp1[lisfac[4]],
                                      mode='markers',
                                      marker=dict(size=2,
                                                  color=temp1['clus'],
                                                  colorbar=dict(title="cluster")))]
                fig = go.Figure(dat3d)

                fig.update_layout(
                    title="cluster-scatter",
                    scene=dict(
                        xaxis_title=lisfac[0],
                        yaxis_title=lisfac[1],
                        zaxis_title=lisfac[4]),)

                fig.show()

                dat3d = [go.Scatter3d(x=temp1[lisfac[0]],
                                      y=temp1[lisfac[2]],
                                      z=temp1[lisfac[3]],
                                      mode='markers',
                                      marker=dict(size=2,
                                                  color=temp1['clus'],
                                                  colorbar=dict(title="cluster")))]
                fig = go.Figure(dat3d)

                fig.update_layout(
                    title="cluster-scatter",
                    scene=dict(
                        xaxis_title=lisfac[0],
                        yaxis_title=lisfac[2],
                        zaxis_title=lisfac[3]),)

                fig.show()

                dat3d = [go.Scatter3d(x=temp1[lisfac[0]],
                                      y=temp1[lisfac[2]],
                                      z=temp1[lisfac[4]],
                                      mode='markers',
                                      marker=dict(size=2, color=temp1['clus'],
                                                  colorbar=dict(title="cluster")))]
                fig = go.Figure(dat3d)

                fig.update_layout(
                    title="cluster-scatter",
                    scene=dict(
                        xaxis_title=lisfac[0],
                        yaxis_title=lisfac[2],
                        zaxis_title=lisfac[4]),)

                fig.show()

                dat3d = [go.Scatter3d(x=temp1[lisfac[0]],
                                      y=temp1[lisfac[3]],
                                      z=temp1[lisfac[4]],
                                      mode='markers',
                                      marker=dict(size=2, color=temp1['clus'],
                                                  colorbar=dict(title="cluster")))]
                fig = go.Figure(dat3d)

                fig.update_layout(
                    title="cluster-scatter",
                    scene=dict(
                        xaxis_title=lisfac[0],
                        yaxis_title=lisfac[3],
                        zaxis_title=lisfac[4]),)

                fig.show()

                dat3d = [go.Scatter3d(x=temp1[lisfac[1]],
                                      y=temp1[lisfac[2]],
                                      z=temp1[lisfac[3]],
                                      mode='markers',
                                      marker=dict(size=2,
                                                  color=temp1['clus'],
                                                  colorbar=dict(title="cluster")))]
                fig = go.Figure(dat3d)

                fig.update_layout(
                    title="cluster-scatter",
                    scene=dict(
                        xaxis_title=lisfac[1],
                        yaxis_title=lisfac[2],
                        zaxis_title=lisfac[3]),)

                fig.show()

                dat3d = [go.Scatter3d(x=temp1[lisfac[1]],
                                      y=temp1[lisfac[2]],
                                      z=temp1[lisfac[4]],
                                      mode='markers',
                                      marker=dict(size=2,
                                                  color=temp1['clus'],
                                                  colorbar=dict(title="cluster")))]
                fig = go.Figure(dat3d)

                fig.update_layout(
                    title="cluster-scatter",
                    scene=dict(
                        xaxis_title=lisfac[1],
                        yaxis_title=lisfac[2],
                        zaxis_title=lisfac[4]),)

                fig.show()

                dat3d = [go.Scatter3d(x=temp1[lisfac[2]],
                                      y=temp1[lisfac[3]],
                                      z=temp1[lisfac[4]],
                                      mode='markers',
                                      marker=dict(size=2,
                                                  color=temp1['clus'],
                                                  colorbar=dict(title="cluster")))]
                fig = go.Figure(dat3d)

                fig.update_layout(
                    title="cluster-scatter",
                    scene=dict(
                        xaxis_title=lisfac[2],
                        yaxis_title=lisfac[3],
                        zaxis_title=lisfac[4]),)

                fig.show()

            # radar

            categories = kmeans_centers.columns.to_list()
            fig = go.Figure()

            for j in kmeans_centers.index:
                fig.add_trace(go.Scatterpolar(
                    r=kmeans_centers.iloc[j],
                    theta=categories,
                    fill='toself',
                    name='cluster_'+str(j)))

            fig.update_layout(
                title="cluster: centroides radar",
                polar=dict(radialaxis=dict(visible=True, range=[0, 1])),
                showlegend=True)
            fig.show()

    # warnings.filterwarnings('default')
    return(data2s2_selec)

In [ ]:
data2inter = seg_kmeans(
    cutD='2018-08-01',
    scalerfitD='2018-08-01',
    cutO=False,
    Standardization=True,
    scal='Standard',
    lisfac=['rec', 'nb_com_ltm', 'pr_mean'],
    ElbowVisualizer=True,
    Sil_ICD_Visualizer=False,
    Sil_ICD_K=[3, 4, 5, 6])

In [ ]:
data2inter = seg_kmeans(
    cutD='2018-08-01',
    scalerfitD='2018-08-01',
    cutO=False,
    Standardization=True,
    scal='Standard',
    lisfac=['rec', 'nb_com_ltm', 'pr_mean'],
    ElbowVisualizer=True,
    Sil_ICD_Visualizer=True,
    Sil_ICD_K=[3, 4, 5, 6])

In [ ]:
data2inter = seg_kmeans(
    cutD='2018-08-01',
    scalerfitD='2018-08-01',
    cutO=False,
    Standardization=True,
    scal='Standard',
    lisfac=['rec', 'nb_com_ltm', 'pr_mean', 'dist_zip'],
    ElbowVisualizer=True,
    Sil_ICD_Visualizer=False,
    Sil_ICD_K=[3, 4, 5, 6])

In [ ]:
data2inter = seg_kmeans(
    cutD='2018-08-01',
    scalerfitD='2018-08-01',
    cutO=False,
    Standardization=True,
    scal='Standard',
    lisfac=['rec', 'nb_com_ltm', 'pr_mean', 'dist_zip'],
    ElbowVisualizer=True,
    Sil_ICD_Visualizer=True,
    Sil_ICD_K=[3, 4, 5, 6])

In [ ]:
data2inter = seg_kmeans(
    cutD='2018-08-01',
    scalerfitD='2018-08-01',
    cutO=False,
    Standardization=True,
    scal='Standard',
    lisfac=['rec', 'nb_com_ltm', 'pr_mean', 'dist_zip', 'note_last'],
    ElbowVisualizer=True,
    Sil_ICD_Visualizer=False,
    Sil_ICD_K=[3, 4, 5, 6])

In [ ]:
data2inter = seg_kmeans(
    cutD='2018-08-01',
    scalerfitD='2018-08-01',
    cutO=False,
    Standardization=True,
    scal='Standard',
    lisfac=['rec', 'nb_com_ltm', 'pr_mean', 'dist_zip', 'note_last'],
    ElbowVisualizer=True,
    Sil_ICD_Visualizer=True,
    Sil_ICD_K=[3, 4, 5, 6])

# DBSCAN

In [ ]:
# DBSCAN algorithm views clusters as areas of HIGH DENSITY
# separated by areas of LOW DENSITY
# clusters found by DBSCAN can be ANY SHAPE, as opposed to k-means
# which assumes that clusters are CONVEX SHAPED
# A cluster is therefore a set of CORE SAMPLES (HIGH DENNITY),
# each close to each other (measured by some distance measure)
# and a set of non-core samples (LOW DESNITY) that are close to a core sample
# (but are not themselves core samples)

# Higher MIN_SAMPLES or lower EPS indicate HIGHER DENSITY necessary
# to form a cluster
# when EPS too SMALL: most data will not be clustered (-1)
# when EPS too large : only one cluster

# MIN_SAMPLES need to be larger when big sample and noise

# CORE SAMPLE = sample in the dataset such that there exist MIN_SAMPLES
# other samples within a distance of EPS, which are NEIGHBOORHOOD
# of the core sample.
# cluster is a set of core samples that can be built by recursively
# taking a core sample, finding all of its neighbors that are core samples,
# finding all of their neighbors that are core samples, and so on
# A cluster also has a set of non-core samples (FRINGES=MARGES)

# implementation:
# the core samples will always be assigned to the same clusters
# the labels of those clusters will depend on the order in which
# those samples are encountered
# the clusters to which non-core samples are assigned can differ
# depending on the data order.
# This would happen when a non-core sample has a distance lower
# than eps to two core samples in different clusters

# OPTICS more memory efficient
from sklearn import metrics
from sklearn.cluster import DBSCAN
res = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


data2mod1 = seg_kmeans(
    cutD='2017-10-01',
    scalerfitD='2017-10-01',
    cutO=False,
    Standardization=True,
    scal='Standard',
    lisfac=['rec', 'nb_com_ltm', 'pr_mean'],
    ElbowVisualizer=False,
    Sil_ICD_Visualizer=False)

for eps_v in np.arange(0.1, 2.1, 0.1).tolist():
    for min_samples_v in [10]+np.arange(100, 600, 100).tolist():
        # eps:The maximum distance between two samples for one
        # to be considered as in the neighborhood of the other
        # eps defaut=0.5
        clustering = DBSCAN(
            eps=eps_v, min_samples=min_samples_v, n_jobs=-1).fit(data2mod1)
        temp = pd.DataFrame(pd.DataFrame(clustering.labels_, columns=[
                            'clus']).value_counts(normalize=True),
                            columns=['pct'])
        temp = temp.reset_index()

        # pour calculer mediane des facteurs standardisés pour le no cluster -1
        tempp = pd.concat([data2mod1, pd.DataFrame(
            clustering.labels_,
            columns=['clus'],
            index=data2mod1.index)],
            axis=1)

        res = res+[[eps_v,
                    min_samples_v,
                    temp.loc[(temp.clus != -1), :].shape[0],
                    temp.loc[(temp.clus == -1), 'pct'].values[0],
                    temp.pct.max(),
                    metrics.silhouette_score(
                        data2mod1, clustering.labels_, metric="euclidean"),
                    metrics.calinski_harabasz_score(
                        data2mod1, clustering.labels_),
                    tempp.loc[tempp.clus == -1].mean()[0],
                    tempp.loc[tempp.clus == -1].mean()[1],
                    tempp.loc[tempp.clus == -1].mean()[2]]]

res = pd.DataFrame(res, columns=['eps', 'min_samples',
                                 'nb_clus', 'pct_no_clus',
                                 'clus_pct_max', 'sil_score', 'cal_score',
                                 'rec', 'com', 'pr'])
res = res.loc[(res.eps > 0), :]

plt.rcParams["figure.figsize"] = [16, 12]
plt.rcParams["figure.autolayout"] = True
f, axes = plt.subplots(3, 3)
sns.scatterplot(data=res, x='eps', y='nb_clus',
                hue='min_samples', ax=axes[0, 0])
sns.scatterplot(data=res, x='eps', y='nb_clus',
                hue='min_samples', ax=axes[0, 1])
sns.scatterplot(data=res, x='eps', y='pct_no_clus',
                hue='min_samples', ax=axes[0, 2])
sns.scatterplot(data=res, x='eps', y='sil_score',
                hue='min_samples', ax=axes[1, 0])
sns.scatterplot(data=res, x='eps', y='cal_score',
                hue='min_samples', ax=axes[1, 1])
sns.scatterplot(data=res, x='eps', y='clus_pct_max',
                hue='min_samples', ax=axes[1, 2])
sns.scatterplot(data=res, x='eps', y='rec',
                hue='min_samples', ax=axes[2, 0])
sns.scatterplot(data=res, x='eps', y='com',
                hue='min_samples', ax=axes[2, 1])
sns.scatterplot(data=res, x='eps', y='pr',
                hue='min_samples', ax=axes[2, 2])

axes[0, 0].set_title(
    'scatterplot-hyperparamètres: nombre de clusters', fontsize=10)
axes[0, 1].set_title(
    'scatterplot-hyperparamètres: nombre de clusters (zoom)', fontsize=10)
axes[0, 1].set_ylim(0, 5)
axes[0, 2].set_title(
    'scatterplot-hyperparamètres: PCT data sans cluster', fontsize=10)

axes[1, 0].set_title(
    'scatterplot-hyperparamètres: silhouette score', fontsize=10)
axes[1, 1].set_title(
    'scatterplot-hyperparamètres: calinski score', fontsize=10)
axes[1, 2].set_title(
    'scatterplot-hyperparamètres: PCT MAX data dans 1 cluster', fontsize=10)

axes[2, 0].set_title(
    'hyperparamètres: recence standard median pour clus -1 ', fontsize=10)
axes[2, 1].set_title(
    'hyperparamètres: nb com standard median pour clus -1', fontsize=10)
axes[2, 2].set_title(
    'hyperparamètres: prix standard median pour clus -1', fontsize=10)

plt.show()

# AGGLOMERATIVE VS KMEANS

In [35]:
# Hierarchical clustering
# general family of clustering algorithms that build nested clusters
# by merging or splitting them successively
# This hierarchy of clusters is represented as a tree (or dendrogram)

#  AgglomerativeClustering object performs a hierarchical clustering
# using a bottom up approach.
# possibility of adding connectivity constraints
# computationally expensive
# metric used for the merge strategy:
# Ward : minimize squared differences within clusters ->
# variance minimising approcah (like k means)
# gives the most regular sizes
# but no variety of distance
# complete linkage -> minimize distance max between 2 clusters
# average linkage -> minimize distance mean between 2 clusters
# (good choice for non euclidian metric)
# single linkage ->minimize distance min between 2 clusters
# (worst startegy but time efficient)

# Varying the metric (only for complete/average/single)
# Euclidean distance (l2),
# Manhattan distance (or Cityblock, or l1:
# l1 good for sparse feature (many 0)),
# cosine distance (invariant to global scalings)
# any precomputed affinity matrix

# guidelines for choosing a metric is to use one that maximizes
# the distance between samples in different classes,
# and minimizes that within each class.

import time
from sklearn.metrics.cluster import calinski_harabasz_score,\
    davies_bouldin_score, silhouette_score
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import adjusted_rand_score,\
completeness_score, normalized_mutual_info_score, fowlkes_mallows_score

data2mod = seg_kmeans(
    cutD='2017-10-01',
    scalerfitD='2017-10-01',
    cutO=False,
    Standardization=True,
    scal='Standard',
    lisfac=['rec', 'nb_com_ltm', 'pr_mean'],
    ElbowVisualizer=False,
    Sil_ICD_Visualizer=False)

# mod_asc=AgglomerativeClustering(
#    n_clusters=4,
#    affinity='euclidean', # “euclidean”, “l1”, “l2”, “manhattan”,
# “cosine”, or “precomputed” #if ward only euclidean
#    linkage='ward', #{‘ward’, ‘complete’, ‘average’, ‘single’},
# default=’ward’
#    compute_distances='False') #usefull for dendrogram

j = 0
for i in [2, 3, 4, 5, 6, 7]:
    j = j+1
    # init
    mod_asc_1 = AgglomerativeClustering(n_clusters=i, linkage='ward')
    mod_asc_2 = AgglomerativeClustering(n_clusters=i, linkage='complete')
    mod_asc_3 = AgglomerativeClustering(n_clusters=i, linkage='average')
    mod_asc_4 = AgglomerativeClustering(n_clusters=i, linkage='single')
    mod_km = KMeans(i)
    # fit
    start = time.time()
    mod_asc_1.fit(data2mod)
    end1 = time.time()-start
    start = time.time()
    mod_asc_2.fit(data2mod)
    end2 = time.time()-start
    start = time.time()
    mod_asc_3.fit(data2mod)
    end3 = time.time()-start
    start = time.time()
    mod_asc_4.fit(data2mod)
    end4 = time.time()-start
    start = time.time()
    mod_km.fit(data2mod)
    end0 = time.time()-start

    # similitarity score
    ari_1 = adjusted_rand_score(mod_asc_1.labels_, mod_km.labels_)
    ari_2 = adjusted_rand_score(mod_asc_2.labels_, mod_km.labels_)
    ari_3 = adjusted_rand_score(mod_asc_3.labels_, mod_km.labels_)
    ari_4 = adjusted_rand_score(mod_asc_4.labels_, mod_km.labels_)

    nmi_1 = normalized_mutual_info_score(mod_asc_1.labels_, mod_km.labels_)
    nmi_2 = normalized_mutual_info_score(mod_asc_2.labels_, mod_km.labels_)
    nmi_3 = normalized_mutual_info_score(mod_asc_3.labels_, mod_km.labels_)
    nmi_4 = normalized_mutual_info_score(mod_asc_4.labels_, mod_km.labels_)

    fms_1 = fowlkes_mallows_score(mod_asc_1.labels_, mod_km.labels_)
    fms_2 = fowlkes_mallows_score(mod_asc_2.labels_, mod_km.labels_)
    fms_3 = fowlkes_mallows_score(mod_asc_3.labels_, mod_km.labels_)
    fms_4 = fowlkes_mallows_score(mod_asc_4.labels_, mod_km.labels_)
    # dispersion score
    # The best value is 1 and the worst value is -1
    sil_1 = metrics.silhouette_score(data2mod, mod_asc_1.labels_)
    sil_2 = metrics.silhouette_score(data2mod, mod_asc_2.labels_)
    sil_3 = metrics.silhouette_score(data2mod, mod_asc_3.labels_)
    sil_4 = metrics.silhouette_score(data2mod, mod_asc_4.labels_)
    sil_0 = metrics.silhouette_score(data2mod, mod_km.labels_)
    # The score is defined as ratio between the
    # within-cluster dispersion and the between-cluster dispersion
    cal_1 = metrics.calinski_harabasz_score(data2mod, mod_asc_1.labels_)
    cal_2 = metrics.calinski_harabasz_score(data2mod, mod_asc_2.labels_)
    cal_3 = metrics.calinski_harabasz_score(data2mod, mod_asc_3.labels_)
    cal_4 = metrics.calinski_harabasz_score(data2mod, mod_asc_4.labels_)
    cal_0 = metrics.calinski_harabasz_score(data2mod, mod_km.labels_)

    # The minimum score is zero,
    # with lower values indicating better clustering.
    db_1 = metrics.davies_bouldin_score(data2mod, mod_asc_1.labels_)
    db_2 = metrics.davies_bouldin_score(data2mod, mod_asc_2.labels_)
    db_3 = metrics.davies_bouldin_score(data2mod, mod_asc_3.labels_)
    db_4 = metrics.davies_bouldin_score(data2mod, mod_asc_4.labels_)
    db_0 = metrics.davies_bouldin_score(data2mod, mod_km.labels_)

    res = [[i,
            end0, end1, end2, end3, end4,
            sil_0, sil_1, sil_2, sil_3, sil_4,
            cal_0, cal_1, cal_2, cal_3, cal_4,
            db_0, db_1, db_2, db_3, db_4,
            ari_1, ari_2, ari_3, ari_4,
            nmi_1, nmi_2, nmi_3, nmi_4,
            fms_1, fms_2, fms_3, fms_4]]

    if j == 1:
        resul = res
    else:
        resul = resul+res

# graphiques de comparaison
lisnam = ['nb_clus',
          't_km', 't_ward', 't_comp', 't_ave', 't_sing',
          'sil_km', 'sil_ward', 'sil_comp', 'sil_ave', 'sil_sing',
          'cal_km', 'cal_ward', 'cal_comp', 'cal_ave', 'cal_sing',
          'db_km', 'db_ward', 'db_comp', 'db_ave', 'db_sing',
          'ari_ward', 'ari_comp', 'ari_ave', 'ari_sing',
          'nmi_ward', 'nmi_comp', 'nmi_ave', 'nmi_sing',
          'fms_ward', 'fms_comp', 'fms_ave', 'fms_sing']
resultat = pd.DataFrame(resul, columns=lisnam)

In [ ]:
plt.rcParams["figure.figsize"] = [16, 10]
plt.rcParams["figure.autolayout"] = True
fig, axes = plt.subplots(2, 3)

axes[0, 0].plot(resultat.nb_clus, resultat.t_km, marker='o', color='b')
axes[0, 0].plot(resultat.nb_clus, resultat.t_ward, marker='+', color='g')
axes[0, 0].plot(resultat.nb_clus, resultat.t_comp, marker='x', color='r')
axes[0, 0].plot(resultat.nb_clus, resultat.t_ave, marker='s', color='c')
axes[0, 0].plot(resultat.nb_clus, resultat.t_sing, marker="*", color='m')
axes[0, 0].set_title('Time comparison')
axes[0, 0].set_xlabel('cluster nb')
axes[0, 0].set_ylabel('time')
axes[0, 0].legend(['kmeans', 'agglo_ward', 'agglo_comp',
                  'agglo_mean', 'agglo_single'])

axes[0, 1].plot(resultat.nb_clus, resultat.sil_km, marker='o', color='b')
axes[0, 1].plot(resultat.nb_clus, resultat.sil_ward, marker='+', color='g')
axes[0, 1].plot(resultat.nb_clus, resultat.sil_comp, marker='x', color='r')
axes[0, 1].plot(resultat.nb_clus, resultat.sil_ave, marker='s', color='c')
axes[0, 1].plot(resultat.nb_clus, resultat.sil_sing, marker="*", color='m')
axes[0, 1].set_title('Silhouette score comparison')
axes[0, 1].set_xlabel('cluster nb')
axes[0, 1].set_ylabel('score')
axes[0, 1].legend(['kmeans', 'agglo_ward', 'agglo_comp',
                  'agglo_mean', 'agglo_single'])

axes[0, 2].plot(resultat.nb_clus, resultat.cal_km, marker='o', color='b')
axes[0, 2].plot(resultat.nb_clus, resultat.cal_ward, marker='+', color='g')
axes[0, 2].plot(resultat.nb_clus, resultat.cal_comp, marker='x', color='r')
axes[0, 2].plot(resultat.nb_clus, resultat.cal_ave, marker='s', color='c')
axes[0, 2].plot(resultat.nb_clus, resultat.cal_sing, marker="*", color='m')
axes[0, 2].set_title('Calinski score comparison')
axes[0, 2].set_xlabel('cluster nb')
axes[0, 2].set_ylabel('score')
axes[0, 2].legend(['kmeans', 'agglo_ward', 'agglo_comp',
                  'agglo_mean', 'agglo_single'])


axes[1, 0].plot(resultat.nb_clus, resultat.ari_ward, marker='+', color='g')
axes[1, 0].plot(resultat.nb_clus, resultat.ari_comp, marker='x', color='r')
axes[1, 0].plot(resultat.nb_clus, resultat.ari_ave, marker='s', color='c')
axes[1, 0].plot(resultat.nb_clus, resultat.ari_sing, marker="*", color='m')
axes[1, 0].set_title('adjusted rand index vs kmeans')
axes[1, 0].set_xlabel('cluster nb')
axes[1, 0].set_ylabel('score')
axes[1, 0].legend(['agglo_ward', 'agglo_comp', 'agglo_mean', 'agglo_single'])


axes[1, 1].plot(resultat.nb_clus, resultat.nmi_ward, marker='+', color='g')
axes[1, 1].plot(resultat.nb_clus, resultat.nmi_comp, marker='x', color='r')
axes[1, 1].plot(resultat.nb_clus, resultat.nmi_ave, marker='s', color='c')
axes[1, 1].plot(resultat.nb_clus, resultat.nmi_sing, marker="*", color='m')
axes[1, 1].set_title('normalized mutual info vs kmeans ')
axes[1, 1].set_xlabel('cluster nb')
axes[1, 1].set_ylabel('score')
axes[1, 1].legend(['agglo_ward', 'agglo_comp', 'agglo_mean', 'agglo_single'])


axes[1, 2].plot(resultat.nb_clus, resultat.fms_ward, marker='+', color='g')
axes[1, 2].plot(resultat.nb_clus, resultat.fms_comp, marker='x', color='r')
axes[1, 2].plot(resultat.nb_clus, resultat.fms_ave, marker='s', color='c')
axes[1, 2].plot(resultat.nb_clus, resultat.fms_sing, marker="*", color='m')
axes[1, 2].set_title('fowlkes mallows vs kmeans')
axes[1, 2].set_xlabel('cluster nb')
axes[1, 2].set_ylabel('score')
axes[1, 2].legend(['agglo_ward', 'agglo_comp', 'agglo_mean', 'agglo_single'])

## stabilité temporelle

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import adjusted_rand_score,\
    completeness_score, normalized_mutual_info_score, fowlkes_mallows_score
from yellowbrick.cluster import KElbowVisualizer,\
    SilhouetteVisualizer, InterclusterDistance
import time
from scipy import stats


def seg_stab(
        tickweek=1,
        mod=KMeans(4),
        datinit='2017-10-01',
        fac=['rec', 'nb_com_ltm', 'pr_mean'],
        scaltype='Standard'):
    
    """kmeans stability.
        
        permet de tracer L'adjusted rand index et d autres metrics entre
        le predict du modèle initial et le fit d'un nouveau modèle
    
        tickweek: tick en nombre de semaines
        mod: type de modele - kemans uniquement pour le moment
        datinit: cut de selction de periode pour initialisation du modèle
        fac: liste de features
        scaltype: type de standardisation : 'Standard'
    """

    start = time.time()
    j = -tickweek
    k = -1
    # plus facile de faire un tick semaine qu'un type mois
    # qui necessite une meilleure gestion (mois differents)

    # initialisation du fichier client à T0=datinit
    # (pour le scaler et la periode)
    data2mod_init = seg_kmeans(
        cutD=datinit,
        scalerfitD=datinit,
        cutO=False,
        Standardization=True,
        scal=scaltype,
        lisfac=fac,
        ElbowVisualizer=False,
        Sil_ICD_Visualizer=False)
    # modèle intial
    model_init = mod
    model_init.fit(data2mod_init)

    for i in np.arange(np.datetime64(datinit),
                       np.datetime64('2018-09-01'),
                       np.timedelta64(tickweek, 'W')):
        j = j+tickweek
        k = k+1

        # time to string
        l = pd.to_datetime(str(i)).strftime('%Y-%m-%d')

        # fichier client standardise intialement
        data2mod1 = seg_kmeans(
            cutD=l,
            scalerfitD=datinit,
            cutO=False,
            Standardization=True,
            scal=scaltype,
            lisfac=fac,
            ElbowVisualizer=False,
            Sil_ICD_Visualizer=False)

        # fichier client standardisé ulterieurement
        data2mod2 = seg_kmeans(
            cutD=l,
            scalerfitD=l,
            cutO=False,
            Standardization=True,
            scal=scaltype,
            lisfac=fac,
            ElbowVisualizer=False,
            Sil_ICD_Visualizer=False)

        # prediction à partir du modèle intial
        mod_labels1 = pd.DataFrame(model_init.predict(data2mod1), columns=[
                                   'A'+str(j)], index=data2mod1.index)

        # fit d'un nouveau modèle uleterieur
        model_ult = mod
        model_ult.fit(data2mod2)
        mod_labels2 = pd.DataFrame(model_ult.labels_, columns=[
                                   'B'+str(j)], index=data2mod2.index)

        mod_labels = mod_labels1.join(mod_labels2)

        if k == 0:
            ari = [[j, adjusted_rand_score(
                mod_labels['A'+str(j)], mod_labels['B'+str(j)])]]
            cs = [[j, completeness_score(
                mod_labels['A'+str(j)], mod_labels['B'+str(j)])]]
            nmi = [[j, normalized_mutual_info_score(
                mod_labels['A'+str(j)], mod_labels['B'+str(j)])]]
            fms = [[j, fowlkes_mallows_score(
                mod_labels['A'+str(j)], mod_labels['B'+str(j)])]]

            nbc = [[j, mod_labels.shape[0]]]
        else:
            ari = ari + \
                [[j, adjusted_rand_score(
                    mod_labels['A'+str(j)], mod_labels['B'+str(j)])]]
            cs = cs + \
                [[j, completeness_score(
                    mod_labels['A'+str(j)], mod_labels['B'+str(j)])]]
            nmi = nmi + \
                [[j, normalized_mutual_info_score(
                    mod_labels['A'+str(j)], mod_labels['B'+str(j)])]]
            fms = fms + \
                [[j, fowlkes_mallows_score(
                    mod_labels['A'+str(j)], mod_labels['B'+str(j)])]]

            nbc = nbc+[[j, mod_labels.shape[0]]]
            # le rand score est une generalisation de l'accuracy binaire
            # l'adjusted normalise en comparant a un rand score aleatoire

    ari = pd.DataFrame(ari, columns=['weeks', 'ari'])
    cs = pd.DataFrame(cs, columns=['weeks', 'cs'])
    nmi = pd.DataFrame(nmi, columns=['weeks', 'nmi'])
    fms = pd.DataFrame(fms, columns=['weeks', 'fms'])

    nbc = pd.DataFrame(nbc, columns=['weeks', 'nbc'])
    nbc['pcc'] = 100*((nbc['nbc']-nbc['nbc'][0])/nbc['nbc'][0])

    plt.rcParams["figure.figsize"] = [16, 5]
    plt.rcParams["figure.autolayout"] = True
    fig, axes = plt.subplots(1, 2)

    axes[0].plot(ari.weeks, ari.ari, marker='o', color='b')
    axes[0].plot(ari.weeks, cs.cs, marker='+', color='g')
    axes[0].plot(ari.weeks, nmi.nmi, marker='x', color='r')
    axes[0].plot(ari.weeks, fms.fms, marker='s', color='c')

    axes[0].set_title(
        'Smiliarity score between label mod and n weeks lagged labels')
    axes[0].set_xlabel('weeks')
    axes[0].set_ylabel('Similarity score')

    axes[0].legend(['Adjusted Rand Index', 'Completeness',
                   'Normalized Mutual Info', 'Fowlkes Mallows'])

    axes[1].plot(nbc.weeks, nbc.pcc, marker='o', color='b',
                 label='croissance de la clientèle')
    axes[1].set_title('croissance du nombre de clients')
    axes[1].set_xlabel('weeks')
    axes[1].set_ylabel('Pourcentage')
    plt.legend()
    plt.show()

    end = time.time()
    print('computation time for', k, 'iterations:', {end - start})

    return


seg_stab()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import adjusted_rand_score,\
    completeness_score, normalized_mutual_info_score, fowlkes_mallows_score
from yellowbrick.cluster import KElbowVisualizer,\
    SilhouetteVisualizer, InterclusterDistance
import time
from scipy import stats


def seg_stab_research(
        tickweek=1,
        mod=KMeans(6),
        datinit='2017-10-01',
        fac=['rec', 'nb_com_ltm', 'pr_mean'],
        scaltype='Standard',
        metric='ARI',
        seuil=0.8):
    
    """kmeans stability research.
        
        permet de trouver le temps nécéssaire entre 2 remodélisations.
        Pour chaque point dans le temps , on observe à partir de quand une
        metrique comme l'adjusted rand index ou autres passe sous un
        certain seuil
        la métrique compare toujours le resultat du  predict du modèle
        initial à t0 et le fit d'un nouveau modèle de 1 à N tickweek aprés
        t0 (temps du modèle initial)
        t0 va varier jusqu'a la fin de la periode disponible
    
        tickweek: tick en nombre de semaines
        mod: type de modele - kemans uniquement pour le moment
        datinit: premier t0
        fac: liste de features
        scaltype: type de standardisation : 'Standard'
        metric: 
            'ARI':adjusted_rand_score
            'CS':completeness_score
            'NMI':normalized_mutual_info_score
            'FMS':fowlkes_mallows_score
        seuil: seuil de la metrique sous laquelle il doit y avoir
        remodelisation

    """

    periode = np.arange(np.datetime64(datinit), np.datetime64(
        '2018-09-01'), np.timedelta64(tickweek, 'W'))  # periode
    s = periode.size  # taille de l'array periode initiale
    res = [[-1, datinit, datinit, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
    print(s, 'iterations')

    flagvertical = 100

    # iteration de recherche
    for iteration in range(0, s):
        start = time.time()  # time
        # initialisation
        j = -tickweek
        k = -1
        score = 1  # initialisation du score à 1 > seuil
        flag = False

        m = pd.to_datetime(str(periode[0])).strftime(
            '%Y-%m-%d')  # initialisation de la date de scaler
        # toujours la premiere car on on raccourci
        # par le debut le range periode

        # initialisation du fichier client à T0=datinit
        # (pour le scaler et la periode)
        data2mod_init = seg_kmeans(
            cutD=m,
            scalerfitD=m,
            cutO=False,
            Standardization=True,
            scal=scaltype,
            lisfac=fac,
            ElbowVisualizer=False,
            Sil_ICD_Visualizer=False)
        # modèle intial
        model_init = mod
        model_init.fit(data2mod_init)

        # periode (range de date)
        for i in periode:
            j = j+tickweek  # compteur de temps (commence à 0)
            k = k+1  # compteur de periodes testées (commence à 0)

            # selection de la periode à cluster
            # time to string
            l = pd.to_datetime(str(i)).strftime('%Y-%m-%d')

            # fichier client standardise intialement
            data2mod1 = seg_kmeans(
                cutD=l,
                scalerfitD=m,
                cutO=False,
                Standardization=True,
                scal=scaltype,
                lisfac=fac,
                ElbowVisualizer=False,
                Sil_ICD_Visualizer=False)

            # fichier client standardise ulterieurement
            data2mod2 = seg_kmeans(
                cutD=l,
                scalerfitD=l,
                cutO=False,
                Standardization=True,
                scal=scaltype,
                lisfac=fac,
                ElbowVisualizer=False,
                Sil_ICD_Visualizer=False)

            # prediction à partir du modèle intial
            mod_labels1 = pd.DataFrame(model_init.predict(data2mod1),
                                       columns=[
                                       'A'+str(j)], index=data2mod1.index)

            # fit d'un nouveau modèle uleterieur
            model_ult = mod
            model_ult.fit(data2mod2)
            mod_labels2 = pd.DataFrame(model_ult.labels_,
                                       columns=[
                                           'B'+str(j)], index=data2mod2.index)

            mod_labels = mod_labels1.join(mod_labels2)

            if k == 0:
                nbci = mod_labels.shape[0]

            score1 = adjusted_rand_score(
                mod_labels['A'+str(j)], mod_labels['B'+str(j)])
            score2 = completeness_score(
                mod_labels['A'+str(j)], mod_labels['B'+str(j)])
            score3 = normalized_mutual_info_score(
                mod_labels['A'+str(j)], mod_labels['B'+str(j)])
            score4 = fowlkes_mallows_score(
                mod_labels['A'+str(j)], mod_labels['B'+str(j)])

            if metric == 'ARI':
                score = score1
            elif metric == 'CS':
                score = score2
            elif metric == 'NMI':
                score = score3
            elif metric == 'FMS':
                score = score4

            if score <= seuil:
                res = res+[[iteration, m, l, j, k, seuil, score1,
                            score2,
                            score3, score4, mod_labels.shape[0], nbci]]
                flag = True
                break  # pour sortir de la 2ème boucle de recherche

        # on cut la premiere valeur du array periode
        periode = np.delete(periode, 0)
        # on rempli res si seuil non franchi
        if flag == False:
            res = res+[[iteration, m, l, np.nan, np.nan, seuil, score1,
                        score2, score3, score4, mod_labels.shape[0], nbci]]
            # pour ligne verticale de seuil non atteint
            if flagvertical == 100:
                flagvertical = iteration-1

        end = time.time()
        if iteration == 0:
            print('computation time pour une iteration:', {end - start})
        # print('iterations:',iteration)

    res = pd.DataFrame(res, columns=['iter', 'dat_in', 'dat_out', 'weeks',
                       'tickweek', 'seuil_'+str(metric), 'ARI',
                                     'CS', 'NMI', 'FMS', 'nbc', 'nbci'])
    res['pcc'] = 100*((res['nbc']-res['nbci'])/res['nbci'])
    # on retire la premiere ligne d'initialisation
    res = res.loc[(res.iter > -1), :]

    plt.rcParams["figure.figsize"] = [16, 5]
    plt.rcParams["figure.autolayout"] = True
    fig, axes = plt.subplots(1, 2)

    slope, intercept, r_value, pv, se =\
        stats.linregress(res.loc[res.weeks.notna(
        ), :]['iter'].to_numpy(),
            res.loc[res.weeks.notna(), :]['weeks'].to_numpy())
    sns.regplot(x="iter", y="weeks", data=res, ax=axes[0], ci=99,
                label="y={0:.1f}x+{1:.1f}".format(slope, intercept)).legend(
        loc="best")
    # axes[0].plot(res.iter,res.weeks,'s')
    axes[0].set_title(
        'Nombre de semaines avant remodélisation: seuil_'+str(metric) +
        ':'+str(seuil))
    axes[0].set_xlabel('itérations')
    axes[0].set_ylabel('weeks')

    axes[1].plot(res.iter, res.nbci, marker='o', color='b')
    axes[1].plot(res.iter, res.nbc, marker='+', color='g')
    # , label='seuil non atteint')
    axes[1].axvline(flagvertical, 0, 1, linestyle='--', color='r')
    axes[1].set_title('nombre de clients')
    axes[1].set_xlabel('itérations')
    axes[1].set_ylabel('clientèle')
    axes[1].legend(['nb client initial',
                   'nb client seuil ou fin de periode',
                    'zone de seuil non atteint'])
    plt.legend
    plt.show()

    fig, axes = plt.subplots(1, 2)
    axes[0].plot(res.iter, res['seuil_'+str(metric)],
                 linestyle='--', color='m')
    axes[0].plot(res.iter, res.ARI, marker='o', color='b')
    axes[0].plot(res.iter, res.CS, marker='+', color='g')
    axes[0].plot(res.iter, res.NMI, marker='x', color='r')
    axes[0].plot(res.iter, res.FMS, marker='s', color='c')
    # , label='seuil non atteint')
    axes[0].axvline(flagvertical, 0, 1, linestyle='--', color='r')

    axes[0].set_title('Les scores avant remodélisation')
    axes[0].set_xlabel('iterations')
    axes[0].set_ylabel('Similarity score')
    axes[0].legend(['seuil_'+str(metric), 'Adjusted Rand Index',
                    'Completeness',
                   'Normalized Mutual Info', 'Fowlkes Mallows',
                    'zone de seuil non atteint'])

    axes[1].plot(res.iter, res.pcc, marker='o',
                 color='b', label='croissance clientele')
    axes[1].axvline(flagvertical, 0, 1, linestyle='--',
                    color='r', label='zone de seuil non atteint')
    axes[1].set_title(
        'croissance de la clientèle correspondant au seuil_'+str(metric))
    axes[1].set_xlabel('itérations')
    axes[1].set_ylabel('croissance de clientèle')

    plt.legend()
    plt.show()

    return(res)


#res1 = seg_stab_research(tickweek=1, mod=KMeans(3))
res2 = seg_stab_research(tickweek=1, mod=KMeans(4))
res3 = seg_stab_research(tickweek=1, mod=KMeans(5))